### using shared memory object previously created to run OnACID

In [1]:
import numpy as np
from multiprocessing import shared_memory

In [2]:
# Attach to the existing shared memory block
existing_shm = shared_memory.SharedMemory(name="shared_mem")

c = np.ndarray((250,60,80), dtype=np.int16, buffer=existing_shm.buf)

array([[1534, 1243,  939, ..., 1000, 1009,  905],
       [ 926, 1041, 1397, ..., 1350, 1068, 1354],
       [ 842,  743,  689, ..., 1192, 1242, 1318],
       ...,
       [1069,  743,  637, ...,  371,  820, 1139],
       [1309, 1144, 1079, ...,  777, 1023,  934],
       [ 950, 1011,  873, ...,  706,  947,  735]], dtype=int16)

In [ ]:
if platform in ('linux', 'darwin'):
    """ This code assumes you are running on a Unix-based computing system (like Linux or macOS)
        that allows to start a Process uing a fork() call, NOT Windows. For the latter see the else call below
        https://docs.python.org/3.7/library/multiprocessing.html#the-spawn-and-forkserver-start-methods"""
    q = Queue()
    cnm2.t_wait = []
    cnm2.t_motion = []
    cnm2.t_fit = []
    realtime = []

    def append_to_queue(q, init_batch, T, fr):
        t_start = time()
        for t in range(init_batch, T):
            # read frame and append to queue
            frame = next(iterator)
            q.put(frame)
            sleep(max(0, (t+1-init_batch)/fr - time()+t_start))

    producer = Process(target=append_to_queue, args=(q,init_batch,T,fr))
    producer.start()

    # process first frame
    t = init_batch
    # read form queue (wait for next frame if empty)
    t0=time()
    frame = q.get()
    cnm2.t_wait.append(time()-t0)
    t_start = time() + (1-init_batch)/fr
    # motion correct
    t0 = time()
    frame = cnm2.mc_next(t, frame)
    cnm2.t_motion.append(time()-t0)
    # fit
    t0 = time()
    cnm2.fit_next(t, frame.ravel(order='F'))
    cnm2.t_fit.append(time()-t0)
    rt = time() <= t_start + t/fr
    realtime.append(rt)

    # process remaining frames
    for t in range(init_batch+1, T):
        # read form queue (wait for next frame if empty)
        t0=time()
        frame = q.get()
        cnm2.t_wait.append(time()-t0)
        # motion correct
        t0 = time()
        frame = cnm2.mc_next(t, frame)
        cnm2.t_motion.append(time()-t0)
        # fit
        t0 = time()
        cnm2.fit_next(t, frame.ravel(order='F'))
        cnm2.t_fit.append(time()-t0)
        rt = time() <= t_start + t/fr
        realtime.append(rt)
        # add code to display whatever you want in order to guide the closed-loop experiment below
        print('Realtime: ' + ("\x1b[32mTrue\x1b[0m" if rt else "\x1b[31mFalse\x1b[0m"), end="  \r", flush=True)